# Character Level RNN using LSTM cells.

- Trained on 1MB of Shakespeare.
- Outputs "fake" Shakespeare.

Much comes from a [Keras example](https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py).

## Setup Environment

- Import Keras
- Open up the Shakespeare corpus
- Give each leter an index and create dictionaries to translate from index to character.

In [1]:
## Much borrowed from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import load_model
import numpy as np
import random
import sys

#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
# text = open("tiny-shakespeare.txt").read().lower()
text = open("startrekepisodes.txt").read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
vocabulary_size = len(chars)
print('total chars:', vocabulary_size)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Using TensorFlow backend.


corpus length: 12106
total chars: 55


## Setup Training Data

- Cut up the corpus into sequences of 40 characters.
- Change indexes into "one-hot" vector encodings.

In [2]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

X = np.zeros((len(sentences), maxlen), dtype=int)
y = np.zeros((len(sentences), vocabulary_size), dtype=np.bool)

for i in range(len(sentences)):
    X[i] = np.array(map((lambda x: char_indices[x]), sentences[i]))
    y[i, char_indices[next_chars[i]]] = True
print("Done converting y to one-hot.")
print("Done preparing training corpus, shapes of sets are:")
print("X shape: " + str(X.shape))
print("y shape: " + str(y.shape))
print("Vocabulary of characters:", vocabulary_size)

nb sequences: 4019
Done converting y to one-hot.
Done preparing training corpus, shapes of sets are:
X shape: (4019, 50)
y shape: (4019, 55)
Vocabulary of characters: 55


## Model

- Model has one hidden layer of 128 LSTM cells.
- Input layer is an Embedding to convert from indices to a vector encoding automatically (common trick - but does it work?)

In [4]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(Embedding(vocabulary_size, 128, input_length=maxlen))
model.add(LSTM(128))
model.add(Dense(vocabulary_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
model.summary()

# model.add(LSTM(128, return_sequences=True))
#model.add(Dropout(0.5))
#model.add(LSTM(128, return_sequences=True))
#model.add(Dropout(0.5))

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 128)           7040      
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 55)                7095      
_________________________________________________________________
activation_1 (Activation)    (None, 55)                0         
Total params: 145,719.0
Trainable params: 145,719.0
Non-trainable params: 0.0
_________________________________________________________________


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1, 128)            7040      
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 55)                7095      
_________________________________________________________________
activation_3 (Activation)    (None, 55)                0         
Total params: 145,719.0
Trainable params: 145,719.0
Non-trainable params: 0.0
_________________________________________________________________


## Training

- Train on batches of 128 examples

In [6]:
# Training the Model.
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
4019/4019 [==============================] - 7s - loss: 2.4914     
Epoch 2/20
4019/4019 [==============================] - 7s - loss: 2.2794     
Epoch 3/20
4019/4019 [==============================] - 7s - loss: 2.1213     
Epoch 4/20
4019/4019 [==============================] - 7s - loss: 1.9726     
Epoch 5/20
4019/4019 [==============================] - 7s - loss: 1.8326     
Epoch 6/20
4019/4019 [==============================] - 7s - loss: 1.6728     
Epoch 7/20
4019/4019 [==============================] - 7s - loss: 1.5252     
Epoch 8/20
4019/4019 [==============================] - 8s - loss: 1.3691     
Epoch 9/20
4019/4019 [==============================] - 8s - loss: 1.1833     
Epoch 10/20
4019/4019 [==============================] - 8s - loss: 1.0277     
Epoch 11/20
4019/4019 [==============================] - 8s - loss: 0.8812     
Epoch 12/20
4019/4019 [==============================] - 7s - loss: 0.7468     
Epoch 13/20
4019/4019 [==========================

In [6]:
model.save("keras-shakespeare-LSTM-model-emb.h5")

In [7]:
model.save("keras-startrek-LSTM-model-emb.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 128)           7040      
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 55)                7095      
_________________________________________________________________
activation_1 (Activation)    (None, 55)                0         
Total params: 145,719.0
Trainable params: 145,719.0
Non-trainable params: 0.0
_________________________________________________________________


## Test the Model

- Take a quote then add 400 characters.

### Make a Decoder model

- Needs input length of 1.

In [117]:
model_dec = Sequential()
model_dec.add(Embedding(vocabulary_size, 128, input_length=1, batch_input_shape=(1,1)))
model_dec.add(LSTM(128, stateful=True))
model_dec.add(Dense(vocabulary_size))
model_dec.add(Activation('softmax'))
model_dec.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
model_dec.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (1, 1, 128)               7040      
_________________________________________________________________
lstm_10 (LSTM)               (1, 128)                  131584    
_________________________________________________________________
dense_7 (Dense)              (1, 55)                   7095      
_________________________________________________________________
activation_7 (Activation)    (1, 55)                   0         
Total params: 145,719.0
Trainable params: 145,719.0
Non-trainable params: 0.0
_________________________________________________________________


In [17]:
model_train = load_model("keras-shakespeare-LSTM-model-emb.h5")
model_dec.set_weights(model_train.get_weights())


IOError: Unable to open file (Unable to open file: name = 'keras-shakespeare-lstm-model-emb.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [7]:
model = load_model("keras-shakespeare-LSTM-model-emb.h5")
quote = "Be not afraid of greatness: some are born great, some achieve greatness, and some have greatness thrust upon them."
quote = quote.lower()

def sample_model(seed, length=400):
    generated = ''
    sentence = seed.lower()[:50]
    generated += sentence
    print("Seed: ", generated)
    
    for i in range(length):
        x = np.array(map((lambda x: char_indices[x]), sentence))
        x = np.reshape(x,(1,50))
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char
    print("Generated: ", generated)

sample_model(quote, 1000)

Seed:  be not afraid of greatness: some are born great, s
Generated:  be not afraid of greatness: some are born great, since i have be prepence an it master heaven
the record own therefore the down an an heart
that thee with the dame hearts you to will it the percomes would boldness heart on her have my tower her.

secong shall the capetis:
what he may man;
and the stand an earth,
and i the bantent of the man to how the mannte that a prepenting of how he the dead heart i dienter the prepain of deade and men to be have my grace of the head of my beace of him in heaving head's the heaven and the mack the slempence
the will the live them.

shecinius:
when the from the montague that i that the made.

petruchio:
and in deady.

petruchio:
i with the man must it is the sent the world him diend the drispite well the head the with the shall is heart:
im would the death the warwick.

petruchio:
and it there to with her have my richard an on the come, done thou did a traity heaving in heart the m

In [106]:
indices_char[random.randint(0,vocabulary_size-1)]



'7'

In [119]:
# Load up the prediction model
model_train = load_model("keras-startrek-LSTM-model-emb.h5")
model_dec.set_weights(model_train.get_weights())

In [120]:
# model_train = load_model("keras-startrek-LSTM-model-emb.h5")
# model_dec.set_weights(model_train.get_weights())

def sample_model(seed, model_name, length=400):
    generated = ''
    sentence = seed.lower()[:]
    generated += sentence
    print("Seed: ", generated)
    
    for i in range(length):
        x = np.array(map((lambda x: char_indices[x]), sentence))
        x = np.reshape(x,(1,1))
        preds = model_name.predict(x, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char
    print("Generated: ", generated)

sample_model(indices_char[random.randint(0,vocabulary_size-1)], model_dec, 1000)

Seed:  c
Generated:  churation send of hold
the chation
dusutht wod deather
deatherd
the wriand of the tho wrichture 
the mater sqiin saure the death   
rears fot (par's of erth  par's of dove
strdame
endarprise  
the matin sury the wration
duseather ar therd part one 
brese   
the 3rations 
imate triny ii   
counter  
the armaphuess   
onerpitice   
the warllion   
the sward llong leath 
eedond  
chations 
imate dirt ii
the wroundlllllling ching gameng 
brese ii   
a the dratien   
the empersicy   
the wark for toorrfronser arter shar   
the fold   
afir   
rising of the end of meas
hold sury the the tho wroundiand shild sury the wrophope  
the emperce stiphold  
rears   
farface 
the chusesion 
ching part one 
the wriakerpare in the defor 
undatter sury the wroght (part 2emations 
the thunter ar'sion 
breture  
the death   
rising of the the kerphation
durture sury the chake  
horker   
prosticy 
bod tronge hold  
relf (part 2i   
ymar's of mina   
the warn surviind 
chinise  
the je